<a href="https://colab.research.google.com/github/scrameri/Umweltbiologie/blob/master/Tutorials.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install packages
This Jupyter notebook is based on the programming language [`R`](https://www.r-project.org/about.html) and has been tested on `R` version 3.6.3.

Author: [Simon Crameri](https://peg.ethz.ch/people/person-detail.MTYzNTY1.TGlzdC80MzUsLTczMzM3Mzg4Ng==.html), ETH Zurich, May 2020

In [0]:
need.pckg <- c("GGally","ggplot2","EnvStats","multcomp","factoextra") # needed for this script
if (any(!need.pckg %in% installed.packages())) {
  for (i in need.pckg[!need.pckg %in% installed.packages()]) {
    cat("installing", i, "...\n")
    install.packages(i)
  }
}

### Load libraries

In [0]:
library("GGally") # for ggpairs()
library("ggplot2") # for ggplot()
library("EnvStats") # for qqPlot()
library("multcomp") # for glht()
library("factoextra") # for fviz_pca() plots

***
## Data Exploration


### Read the data

In [0]:
morph <- read.csv("https://raw.githubusercontent.com/scrameri/Umweltbiologie/master/Puzzle_ANOVA_LM.csv")

NOTE: the .csv file uses "," rather than ";" as field separators, which can cause troubles when opening the file in Excel on Mac. You can also save a version separated with ";", but you will then need to set `sep=";"`, because the default argument is `read.csv(file, sep = ',')`. Also have a look at the `read.csv2` function.



***
### Get an overview over the dataset
This dataset contains morphological measurements of eight populations of [*Dianthus carthusianorum*](https://www.infoflora.ch/de/flora/dianthus-carthusianorum.html) taken in Summer 2015 in Wallis (Switzerland). Populations were located in two classes of elevation (4 high, 4 low elevation).

A few facts about [*D. carthusianorum*](https://de.wikipedia.org/wiki/Kart%C3%A4usernelke)
- Karthäuser-Nelke in German, family Caryophyllaceae like *Silene*
- native to Middle Europe, in dry habitats from colline to alpine (introduced to N America) 
- [gynodioecious](https://en.wikipedia.org/wiki/Gynodioecy), meaning that there are female and hermaphroditic individuals
- perennial (or biennial), meaning that plants survive winter as rosettes or seeds
- insect-pollinated
- parasite: fungus [*Microbothryum*](https://fr.wikipedia.org/wiki/Microbotryum) *dianthorum* (anther smut fungus) that causes sterility and is transmitted by pollinators

A few facts about the dataset
- ID: individual ID
- Population: population ID
- Elevation: elevation class
- Date: measurement date
- Infection: whether a plant was [infected](https://upload.wikimedia.org/wikipedia/commons/e/ef/Microbotryum_dianthorum_Dianthus_sp._2019_07_18_05.jpg) or healthy
- Sex: whether a plant was hermaphroditic or female
- Stalk_height: mean stalk height per plant (mm)
- Stalk_count: number of stalks per plant
- Bud_count: number of buds per plant
- Flower_count: number of open flowers per plant
- Flower_diam: flower (corolla) diameter (mm)
- Petal and Sepal lengths and widths (mm)
- Rosette_diam1: diameter of first rosette (mm) - only recorded at high elevation
- Rosette_diam2: diameter of second rosette (mm) - only recorded at high elevation

The experimental design and measurements have been done by [Ursina Walther](https://peg.ethz.ch/people/person-detail.html?persid=158239) (a PhD student in our group). She was interested in studying the evolution of floral traits in this species, especially in relation to the interaction between the plants and their Microbothryum parasite.

Before starting any analysis, always check that the data was read in *correctly*.

In [0]:
str(morph) # lists all variables in the data.frame and their classes
ncol(morph)

In [0]:
dim(morph) # gives both nrow() and ncol() of the dataset


In [0]:
summary(morph$Stalk_height) # gives a summary for one variable

![Question](https://img.icons8.com/flat_round/64/000000/question-mark.png)
 Based on what you now know about the study and the data:
- what are the response variable(s)?
- what are the explanatory variable(s)?

***

### Plot the data
There are various plotting techniques to visualize data. Some people prefer the `R` base graphics, others learn the grammar of graphics implemented in [`ggplot2`](https://ggplot2.tidyverse.org/). For this practical, both approaches can be used. These are the basic functions for the most important plots, shown for `R` base graphics and `ggplot2` graphics:

Use **Scatterplots** to plot two numeric variables (classes `num` for continuous, or `int` for discrete data) against each other. Scatterplots can be more informative if different *symbols* or *colours* for an additional factor, or *sizes* or *transparency* for an additional numeric variable are used.

In [0]:
# base
plot(Sepal_length ~ Petal_length, data = morph)

In [0]:
# ggplot2
ggplot(data = morph, aes(x = Petal_length, y = Sepal_length)) + geom_point()

Use **Boxplots** to plot a numeric variable against a categorical variable (class `factor`). Boxplots are summarizing the data. By default, the box is the *interquartile range* and contains 50% of the data points, while the whiskers extend to "the most extreme data point which is no more than 1.5 times the interquartile range from the box" (see cell 85). If you have less than eight data points per factor level, you might just aswell plot all the data points using `stripchart(Sepal_length ~ Elevation, data = morph, vertical = TRUE)`.

In [0]:
# base
boxplot(Sepal_length ~ Elevation, data = morph, ylim = c(8,20))

# understand interquartile range (iqr): range from 25% to 75% quantiles, contains 50% of the data
iqr1 <- quantile(morph$Sepal_length[morph$Elevation=="high"], c(0.25,0.75))
iqr2 <- quantile(morph$Sepal_length[morph$Elevation=="low"], c(0.25,0.75))

# whiskers extend to the most extreme data point not exceeding 1.5*iqr from the box (by default)
whisker1 <- iqr1 + c(-1.5,1.5)*diff(iqr1)
whisker2 <- iqr2 + c(-1.5,1.5)*diff(iqr2)

segments(0.5, whisker1, 1.5, whisker1, col = "tomato")
segments(1.5, whisker2, 2.5, whisker2, col = "blue")

In [0]:
# ggplot2
ggplot(data = morph, aes(x = Elevation, y = Sepal_length)) + geom_boxplot()

Use **Histograms** to plot a variable's distribution. The `breaks` argument in `hist` and the `bins` argument in `geom_histogram` can be used to fine-tune the binning of values into histogram categories. Try and play around with it.

In [0]:
hist(morph$Sepal_length, breaks = 20)

In [0]:
ggplot(data = morph, aes(x = Sepal_length)) + geom_histogram(bins = 20) 

Use **Barplots** to plot all values of a single variable or a table of counts.

In [0]:
# base
barplot(table(morph$Population))

In [0]:
# ggplot2 (uses count table by default!)
ggplot(data = morph, aes(x = Population)) + geom_bar(stat = "count")

Use **Mosaic Plots** or **Stacked Barplots** to plot *contingency tables* of two categorical variables against each other. In a Mosaic Plot, the relative size of each mosaic is proportional to the number of observations with a given factor level combination. The stacked barplot is slightly different, once shown as stacked *counts* and once as stacked *proportions*.

In [0]:
# base
mosaicplot(table(morph$Elevation,morph$Infection), main = "Mosaic Plot")

In [0]:
# ggplot2 (fill variable determines the stacking)
ggplot(morph, aes(x = Elevation, fill = Infection)) + 
  geom_bar() + 
  ggtitle("Stacked Barplot")
  
ggplot(morph, aes(x = Elevation, fill = Infection)) + 
  geom_bar(position = "fill") + labs(y = "Proportion") +
  ggtitle("Stacked Barplot")

![Question](https://img.icons8.com/flat_round/64/000000/question-mark.png)
- Can you produce a plot of `Petal_length ~ Sepal_length` with points color-coded for `Infection` and size proportional to `Petal_length`?


In [0]:
# Make your plot here

***
### Generate all pairwise plots
#### Use regular expressions
Given that we have 17 variables, plotting all of them against each other by hand would be tedious. For datasets with a moderate number of variables, you can use the `ggpairs` function from the [*GGally*](https://cran.r-project.org/web/packages/GGally/index.html) package to get a graphical overview over many variables at once with a single line of code. The function produces all possible pairwise plots, minimizes redundancy and takes care of factors and numerical variables automatically.

With 17 variables, plotting all against all would lead to too many (289) plots on a single page. Let us therefore subset the variables. You can use `grep` and [**regular expressions**](https://regex101.com/) to find the indices of certain variable names, this often saves code.

In [0]:
# this returns the index of variable names *starting* with 'Flower' (^ specifies the *start*)
grep(pattern = "^Flower_", x = names(morph))

In [0]:
# this returns the index of variable names *ending* with either 'length' or 'width' ($ specifies the *end*, | is a logial <or>)
grep(pattern = "length$|width$", x = names(morph)) 


![Question](https://img.icons8.com/flat_round/64/000000/question-mark.png)
- Can you identify the indices of columns with variables of *fertile* structures (buds, flowers) using a regular expression?

In [0]:
# Create your regular expression here
names(morph)
grep(pattern = "XXX", x = names(morph))

#### print all pairwise plots

In [0]:
print(vars.fertile <- names(morph)[grep(pattern = "^Flower|^Bud|^Petal|^Sepal", x = names(morph))])
print(vars.sterile <- names(morph)[grep(pattern = "^Stalk|^Rosette", x = names(morph))])

The `ggpairs` function prepares the pairwise plots as `gg` class objects. These can then be printed to the plotting device, or saved to a PDF file by enclosing the print commands between the `pdf` and `graphics.off` functions.

It is more reproducible (and therefore more scientific) to save plots with such code rather than by manually exporting a plot from R Studio. The specified `height` and `width` also prevents the saved plot to be smaller or larger depending on your device size.

In [0]:
pairs.fertile <- ggpairs(data = morph, columns = c("Population","Elevation","Infection","Sex", vars.fertile))
pairs.sterile <- ggpairs(data = morph, columns = c("Population","Elevation","Infection","Sex", vars.sterile))

In [0]:
#pdf("Pairsplots.pdf", height = 15, width = 15)
#pairs.sterile # or print(pairs.sterile)
pairs.fertile # or print(pairs.fertile)
#graphics.off()

Does everything look all right?




***
### Transparent data fixes
If there are obvious data errors, do not manually change the original data. Rather think carefully what the issue might be, and if you can be sure enough, make the data changes in the analysis script, such that fixes become **transparent** and **reproducible**. If you cannot reconstruct what happened with an impossible data value, you should set it to `NA` or remove the entire data row in the analysis script.

In [0]:
morph$Flower_diam[morph$Flower_diam > 100] # Dianthus with 185 mm flower diameter is impossible
morph$Flower_diam[morph$Flower_diam > 100] <- morph$Flower_diam[morph$Flower_diam > 100]/10 # if you are sure it is a comma-error
# morph$Flower_diamesser[morph$Flower_diamesser > 100] <- NA # if you cannot be sure what happened


***

### Check the distribution of your variables
Knowing the distribution of your variables is important, especially if you want to create models of them that make certain assumptions on distributions. You have already seen that the **histogram** is a good visualization of a variable's distribution because deviations from a normal distribution become intuitively apparent.

Let us look at `Stalk_count`, which is a count variable and therefore by definition bound by zero on the left, and unbounded on the right. Such *right-skewed* distributions are not normally distributed.

But which statistical distribution is best at describing your data if not a normal distribution? Knowing this would be helpful, e.g. to define an appropriate error distribution in a generalized linear model. **Simulations** can help you to compare your data against data from a specific statistical distribution. The `R` package [*EnvStats*](https://cran.r-project.org/web/packages/EnvStats/index.html) allows for simulation of different probability distributions from your data, and comparison of these against your data in so-called **Q-Q Plots** (Quantile-Quantile plots).

In [0]:
# see ?EnvStats::Distribution.df for a list comprehensive list of continuous and discrete distributions
par(mfrow=c(2,3))
hist(morph$Stalk_count, breaks = 20, col = "tomato")
qqPlot(morph$Stalk_count, distribution = "pois", estimate.params = TRUE, add.line = TRUE, points.col = "tomato")
qqPlot(morph$Stalk_count, distribution = "geom", estimate.params = TRUE, add.line = TRUE, points.col = "tomato")
hist(morph$Flower_diam, breaks = 20, col = "cadetblue")
qqPlot(morph$Flower_diam, distribution = "norm", estimate.params = TRUE, add.line = TRUE, points.col = "cadetblue")
qqPlot(morph$Flower_diam, distribution = "exp", estimate.params = TRUE, add.line = TRUE, points.col = "cadetblue")
par(mfrow=c(1,1))

![Question](https://img.icons8.com/flat_round/64/000000/question-mark.png)
- Which probability distribution is best at describing `Stalk_count`? 
- Which probability distribution is best at describing `Flower_diam`? 
- What does this mean for an ANOVA model `Stalk_count ~ Elevation` fitted to this data?

***

## Analysis of Variance and Linear Models

### The relationship between t-test, ANOVA and linear models

**Analysis of Variance (ANOVA)** using `aov` and (generalized) **linear models** using `lm`or `glm` are closely related to each other and differ mainly in *intent* of analysis and default *presentation* of results. If you read the aov help page, you will see that `aov` actually calls `lm`, so both rely on the same [least squares](https://en.wikipedia.org/wiki/Least_squares) method.


You can think of ANOVA and linear models as extensions of a two-sample [**t-test**](https://en.wikipedia.org/wiki/Student%27s_t-test) that allow for tests of more factors or covariates at once. Remember that the two-sample t-test allows to test the difference in a dependent variable (a.k.a. **response** variable, measure variable, Y variable) between two levels of one factor (a.k.a. **predictor** variable, independent variable, X variable). An example two-sample t-test is the test for difference in human height (the response variable) between two sexes (the predictor variable). 

Let us start with a **one-sample t-test**:

In [0]:
t.test(morph$Sepal_length, mu = 0)

![Question](https://img.icons8.com/flat_round/64/000000/question-mark.png)
- What are the null and alternative hypotheses? 
- What is the test result?

This test can also be accomplished using `lm`:

In [0]:
summary(lm(Sepal_length ~ 1, data = morph))

Let us now extend this to a **two-sample t-test** (one sample at low and one at high elevation):

In [0]:
# complicated syntax, showing the two samples (sepal lengths at high and low elevation)
# t.test(morph$Sepal_length[morph$Elevation == "high"], morph$Sepal_length[morph$Elevation == "low"], data = morph, mu = 0)

# shorter and more elegant formula notation 
t.test(Sepal_length ~ Elevation, data = morph, mu = 0)

Again, the same test can be accomplished using `lm` or `aov`. A `summary` on `aov` will give you the **ANOVA table**, while `lm` will give you **effect size** estimates.

In [0]:
cat("**ANOVA table**\n")
summary(aov(Sepal_length ~ Elevation, data = morph))

cat("\n\n**Effect size estimates**\n")
summary(lm(Sepal_length ~ Elevation, data = morph))

![Question](https://img.icons8.com/flat_round/64/000000/question-mark.png)
- What are the null and alternative hypotheses? 
- What is the estimated effect size of Elevation?
- How do you interpret the Elevation effect estimate intuitively?


***

### One-way ANOVA and simple linear models

The **one-way ANOVA** allows for comparing more then just two groups (i.e. one treatment factor can have more than two levels). 

Tests with more than 2 levels in a predictor variable can have different intents: 

1. test for an *overall* effect of a factor such as `Population` on calyx length
2. test for an effect of a *specific* factor level, for instance a treatment relative to a *control*, including effect size estimates.

If your goal is 1., then you can use `aov`:

In [0]:
summary(aov1 <- aov(Sepal_length ~ Population, data = morph))

If your goal is 2., then you can use `lm`. Note that a `summary` on `lm` does not give you an answer to the hypothesis "Population has an effect on Sepal_length", while a `summary` on `aov` does.

In [0]:
summary(lm1 <- lm(Sepal_length ~ Population, data = morph))

Irrespective of whether you fitted a model with `aov` or `lm`, you can use some handy accessor functions to **extract key model results**.

In [0]:
## model parameter estimates in coef()
print(coef(lm1))

In [0]:
## confidence interval (by default 95%) for model parameter estimates in confint()
print(confint(aov1,  level = 0.95))

In [0]:
# Tukey-Anscombe plot (see paragraph on Residual Analysis)
dfit <- data.frame(resid = resid(lm1), fitted = fitted(lm1))
ggplot(dfit, aes(fitted, resid)) +
  geom_point() +
  geom_smooth(method = "loess", formula = y ~ x) +
  theme_bw()

# Observed values in black, fitted values in red
ggplot(morph, aes(Population, Sepal_length)) + 
  geom_boxplot() + 
  geom_boxplot(aes(Population, fitted(lm1)), col = "tomato", linetype = 2) +
  theme_bw()

![Question](https://img.icons8.com/flat_round/64/000000/question-mark.png)
- Why are there only 7 estimates of population model parameters instead of 8?
- Can you tell from the **confidence intervals** which population(s) are significantly different from the reference population?

***

### Multi-way ANOVA and multiple linear regression
There are many extensions to one-way ANOVA and simple linear models. In `R`, you can use the same functions `aov` and `lm` (or `glm`) and provide them with a more complex model. 

The **Factorial ANOVA** extends the one-way ANOVA to test >1 predictors (each with two or more levels), including interactions.

The **mathematical model** for two predictors without interaction is as follows:
* Y = β<sub>0</sub> + β<sub>1</sub>\*X<sub>1</sub> + β<sub>2</sub>\*X<sub>2</sub> + E ; E ~ norm(0, sd) i.i.d.

Y, X<sub>1</sub>, and X<sub>2</sub> are given. The model fitting consists of estimating distributions for β<sub>0</sub>, β<sub>1</sub> and β<sub>2</sub>
while minimizing the error term E (approximated by the so-called residuals).

The corresponding **model formula** in `R` is as follows (no need to specify intercept and error term):

* `Y ~ X1 + X2`

Three preditors, the second nested in the first:
* `Y ~ X1/X2 + X3`

Three predictors, two of them interacting:
* `Y ~ X1 + X2 + X1:X2 + X3`
* `Y ~ (X1 + X2)^2 + X3`

Three predictors, include all two-way interactions:
* `Y ~ X1 + X2 + X1:X2 + X1:X3 + X2:X3`
* `Y ~ (X1 + X2 + X3)^2`

Three predictors, include all three-way interactions (rarely used):
* `Y ~ (X1 + X2 + X3)^3`


Beware of the use of highly **correlated predictor variables**. If such are present, the inference can be quite wrong when both are included in the model, as shown in the following example.

In [0]:
# simulate dataset with highly correlated x1 and x2 (both correlated with y)
x1 <- rnorm(n = 500, mean = 0, sd = 1)
x2 <- pi + pi*x1 + rnorm(n = 500, mean = 0, sd = 0.5)
y  <- x2 + rnorm(n = 500, mean = 0, sd = 2)

dsim <- data.frame(y = y, x1 = x1, x2 = x2)

# correlation among x1 and x2
cat("correlation between x1 and x2\n")
print(cor(dsim$x1, dsim$x2))

# model summaries
cat("\n\n=>model with only x1 included")
summary(lm(y ~ x1, data = dsim))

cat("\n\n=>nmodel with only x2 included")
summary(lm(y ~ x2, data = dsim))

cat("\n\n=> model with x1 and x2 included (both highly correlated)")
summary(lm(y ~ x1 + x2, data = dsim))

# graph
ggplot(dsim, aes(x1, y, size = x2)) + geom_point() + theme_bw()

![Question](https://img.icons8.com/flat_round/64/000000/question-mark.png)
- What can you do to prevent the above problem?

***

### ANCOVA and GLMs

The **ANCOVA** (Analysis of Covariance) allows for the mixed use of numerical and categorical predictors. **Generalized linear models** (GLMs) extend ANCOVA by allowing for non-normal error distribution (logistic regression, poisson regression, ...):

Logistic regression
* E ~ Binom(k|p,n) i.i.d

Poisson regression
* E ~ Poisson(lambda) i.i.d.

***
### Interactions
Interactions between predictor (explanatory) variables can be present, and including them in a model can *greatly* improve the model fit and hence the reliability of test results.

An **interaction** occurs if the effect of variable A on the response Y *depends* on variable B.

Example:
You want to model reaction time as Y ~ A + B

* Y = time needed to start breaking after an obstacle appears on the street
* A = amount of coffee drunk 
* B = amount of whisky drunk 

A has a negative effect on reaction time (i.e., more coffee, less reaction time), while B has a positive effect on reaction time (i.e, more whisky, more reaction time).

You can increase coffee consumption to shorten reaction time, but this effect will depend on the amount of whisky drunk. That is, the amount of coffee *interacts* with the amount of whisky.


`R` implements some methods to visualize potential interactions. The idea is to visualize the mean (or other summary) value of the `response` Y at different levels of an explanatory variable of interest (`x.factor`), depending on the level of another and potentially interacting explanatory factor (`trace.factor`).

![Question](https://img.icons8.com/flat_round/64/000000/question-mark.png)
We now want to test the effect of `Infection` and `Altitude` on `Petal_length`. Before you fit any model, make sure to plot the data to get a better feeling for it.

In [0]:
## Make a clever plot(s) that shows Petal_legnth, Infection and Altitude

`Petal_length` appears to be lower in some infected individuals, but that does not seem to be the case in low `Elevation` habitats. That is, the effect of `Infection` on `Petal_length` appears to *depend* on `Elevation`. Or in other words: `Infection` and `Elevation` likely *interact*.

The following plot visualizes potential interactions. If the lines cross, the factors strongly interact and you would need to test for the interaction term in any model of `Petal_Length`.

In [0]:
interaction.plot(x.factor = morph$Infection, trace.factor = morph$Elevation, response = morph$Petal_length, 
                 fun = mean, xlab = "Infection", ylab = "Mean sepal length")

![Question](https://img.icons8.com/flat_round/64/000000/question-mark.png)
We now want to formally test for the effect of `Infection` and `Elevation` (including their potential interaction) on `Petal_length`, and estimate the effect sizes.
* which function do you use (`aov` or `lm`)?
* which formula do you use?
* what are the test results?

In [0]:
## Create your model here
summary(mod1 <- lm(Petal_length ~ Infection * Elevation, data = morph))

***

### Check model assumptions via Residual Analysis
Up to now, we have just blindly executed `summary` on `aov` and `lm` fits and interpreted the test results. In practice, this is **not sufficient**.

In order to validate the model, we need to check that the model assumptions are met, or at least not grossly violated.

ANOVA and linear regression models both make the following main **assumptions**:
  
* [**Normality:**](https://en.wikipedia.org/wiki/Normal_distribution) For any fixed value of X, Y is *normally distributed*. This implies that the *residuals* are normally distributed. However, it does not mean that all variables have to be normally distributed (not even within groups).
* [**Homoscedasticity:**](https://en.wikipedia.org/wiki/Homoscedasticity) The *residual variance* is *constant* (the same for any value of X)
* [**Independence:**](https://en.wikipedia.org/wiki/Independence_(probability_theory)) Observations are *independent* of each other (i.e. one measurement is not influenced by another)

If one or more of these assumptions (see this [link](http://r-statistics.co/Assumptions-of-Linear-Regression.html) for more details) are **violated**, you **cannot have trust in your test results** and in any conclusions drawn from them.

You can check most assumptions by applying the generic `plot` function on your model fit. Since this will give you **four diagnostic plots**, it is best to divide the plotting device into four panels using `par`.

The first plot (**Tukey-Anscombe Plot**) is good to check for the assumption of *constant variance* of the residuals. This assumption is violated if there is a funnel-shape ur U-shaped distribution.

The second plot (**Normal Q-Q Plot**) is good to check for the assumption of *normality* of the residuals. If the points follow the diagonal, they follow a normal distribution. If you don't trust your eye, you can also formally test the null hypothesis of normality using a **Shapiro-Wilk normality test** using `shapiro.test` on the model residuals. If that test turns out significant (p < 0.05), it means that the null hypothesis of normality is rejected, and hence that the residuals are not normally distributed.

The **Scale-Location Plot** is similar to the Tukey-Anscombe Plot but plots the square-root of absolute residuals, which allows to see other aspects of residual variance. 

The **Leverage Plot** is good to identify outliers (points that lie outside of Cook's distance). Such outliers could have a strong effect on the model fit and test results and should be investigated further if present.


In [0]:
# Residual analysis produces 4 plots by default.
par(mfrow = c(2,2)) # prepares an plot device layout with 2 columns and 2 rows (4 plots)
plot(mod1)
par(mfrow = c(1,1))

shapiro.test(mod1$resid)

Independence of measurements can be assessed by plotting a response variable's values in their order of measurements, or with a **Serial Correlation Plot** using the `acf` function (assuming the the order of measurements of `Petal_length` in the table corresponds to the order of measurements taken on each `Date`. If the autocorrelation function ACF is within the blue dotted lines for lags > 0, the measurements are not significantly autocorrelated (or dependent).

In [0]:
ggplot(morph, aes(seq_along(Sepal_length), Sepal_length)) + 
  geom_line() + geom_point(aes(color = Infection, shape = Infection)) +
  labs(x = "Order of measurement") +
  facet_wrap(~ Elevation + Date + Population, scales = "free_x") + 
  theme_bw()

acf(resid(mod1))

![Question](https://img.icons8.com/flat_round/64/000000/question-mark.png)
* Are the assumptions of normality, constant variance and independence of the residuals met?


***
### Variable transformations
Apart from interaction effects, transformations of the resonse Y or the numerical predictors X often help to improve model fit, at the expense of interpretability of the fitted model parameters (estimated effect sizes). There are the following commonly used **first-aid transformations**.

- for *positive counts*:               `X <- sqrt(X)`
- for *positive continuous variables*: `X <- log(X)`
- for *fractions* [0,1]:               `X <- asin(sqr(X))`

![Question](https://img.icons8.com/flat_round/64/000000/question-mark.png)
Now try to apply a transformation to the response.
* Does the model fit improve?

In [0]:
## Create your model here

***
### Compare nested models
Complex models are *less parsimonious* (more parameters need to be estimated). If a complex model can be replaced by a simpler model with an equally good fit, one would always **prefer the simpler model**. 

You can formally test two **nested** models (i.e. fitted to the same data, one model with one or more additional model terms) using the `anova` function. If the F test turns out significant, the **larger / complex** model (in our case, the model including the interaction term) is a **better** fit to the data, and should thus be preferred over the smaller / simpler model.

In [0]:
mod.large <- aov(Petal_length ~ Infection * Elevation, data = morph)
mod.small <- aov(Petal_length ~ Infection + Elevation, data = morph)
print(anova(mod.large,mod.small))

![Question](https://img.icons8.com/flat_round/64/000000/question-mark.png)
* Would you choose `mod.large` or `mod.small`?

Also have a look at the `step` function that allows to identify an optimal model among a much more extensive range of possible models. It does so by adding *one* model term to (or remove one model term from) an initial model (`mod.small` in this case). All single term additions and drops that are within the `scope` are tested, and the model with the smallest [AIC](https://en.wikipedia.org/wiki/Akaike_information_criterion) value is kept. These steps are iterated until the AIC does not decrease anymore, and the final model is returned.

In [0]:
mod.best <- step(object = mod.small, 
                 scope = list(lower = Petal_length ~ 1, 
                              upper = Petal_length ~ Infection * Elevation + Elevation/Population), 
                 direction = "both")

par(mfrow = c(2,2))
plot(mod.best)
par(mfrow = c(1,1))

![Question](https://img.icons8.com/flat_round/64/000000/question-mark.png)
* Which model is selected? (don't forget to check the residuals of this model as well!)

***
### Contrasts
A [**contrast**](https://en.wikipedia.org/wiki/Contrast_(statistics)) is a specific hypothesis to be tested. Imagine that you are not only interested in an *overall* difference in `Petal_length` between `Population`s (or put differently: an overall effect of `Population` on `Petal_length`), but you want to test whether a *specific* population is different from another (or a specific treatment different from a control). You can think of them as t-tests carried out between specific factor levels. 

You can use **Tukey's Honest Significant Differences** method to test for differences in the response variable between *all* pairs of factor levels. This method is implemented in the `TukeyHSD` function but can be used as an option in the more flexible `glht` function from the [multcomp](https://cran.r-project.org/web/packages/multcomp/vignettes/multcomp-examples.pdf) package. The drawback here is that with many factor levels, you are carrying out a lot of tests, and not all of them answer your questions.



In [0]:
# fit a model of Petal_length and Population
mod2 <- aov(Petal_length ~ Population, data = morph)

# perform tests for all pairwise comparisons of populations
tukey <- glht(mod2, linfct = mcp(Population = "Tukey"))
# equivalent: tukey <- TukeyHSD(mod2, which = "Population")

# tabular output
summary(tukey)

# graphical output
oldmar <- par()$mar # get default margins
par(mar = c(5.1, 12, 4.1, 2.1)) # give more left margin
plot(tukey)
par(mar = oldmar) # reset margins to default

# residual analysis for mod2
par(mfrow=c(2,2))
plot(mod2)
par(mfrow=c(2,2))

![Question](https://img.icons8.com/flat_round/64/000000/question-mark.png)
* Which population pair shows the largest difference in `Petal_length`?

The `glht` function can do much more. It allows you to control which comparisons (tests) should be carried out. Each comparison is a separate test on the same dataset, therefore we have a **multiple testing problem**. The function `summary` on `glht` objects allows to set which p-value correction method should be used to correct for the multiple testing. The use of `glht` for specific contrasts requires the definition of a **contrast matrix**. It is a bit tricky to set up, but try to understand the following example, and you will be able to set specific contrasts that exactly match your research questions, while accounting for the multiple testing involved. These contrasts can be pairwise comparisons, but can also be more complex, as in the next example.

Let us now test whether `Petal_length` is significatly different between 
- Zeneggen and (Faldumalp and Grengiols)
- Zermatt and Niedergampel
- Zermatt and Zeneggen
- Zeneggen and Unterstalden
- Zeneggen and Grengiols

Follow these *rules* to create the **contrast matrix** *K*
- use sensible rownames for each contrast (will appear in the summary)
- columns correspond to the factor level (in the same order as `levels(<factor>)`, alphabetical by default)
- set the coefficient to `0` if a level is not used in a contrast
- use positive coefficients for levels in the first group and negative coefficients for levels in the second group
- levels in the first and second group *must* add up to 1 and -1, respectively
- each row *must* add up to zero

In [0]:
# define the contrast matrix for specific contrasts of interest
#              first  -  second                 Fal, Gib, Gre, Nie, Sim, Unt, Zen, Zer
K <- rbind("Zeneggen - Faldumalp+Grengiols"= c(-1/2,   0,-1/2,   0,   0,   0,   1,   0),
           "Zermatt - Niedergampel"        = c(   0,   0,   0,  -1,   0,   0,   0,   1),
           "Zermatt - Zeneggen"            = c(   0,   0,   0,   0,   0,   0,  -1,   1),
           "Zeneggen - Unterstalden"       = c(   0,   0,   0,   0,   0,  -1,   1,   0),
           "Zeneggen - Grengiols"          = c(   0,   0,  -1,   0,   0,   0,   1,   0))

# tabular output
# "single-step" is the default p-value correcton method
# The "BH" (Bonferroni-Holm) method is less conservative and more powerful than "bonferroni"
summary(specific <- glht(mod2, linfct = mcp(Population = K)), test = adjusted("single-step"))

# corresponding t-tests for comparison
t.test(morph$Petal_length[morph$Population %in% c("Zeneggen")],
       morph$Petal_length[morph$Population %in% c("Faldumalp","Grengiols")])
t.test(morph$Petal_length[morph$Population %in% "Zermatt"],
       morph$Petal_length[morph$Population %in% "Niedergampel"])
t.test(morph$Petal_length[morph$Population %in% "Zermatt"],
       morph$Petal_length[morph$Population %in% "Zeneggen"])
t.test(morph$Petal_length[morph$Population %in% "Zeneggen"],
       morph$Petal_length[morph$Population %in% "Unterstalden"])
t.test(morph$Petal_length[morph$Population %in% "Zeneggen"],
       morph$Petal_length[morph$Population %in% "Grengiols"])

![Question](https://img.icons8.com/flat_round/64/000000/question-mark.png)
Compare results of the specific contrasts to the TukeyHSD results and the t-test results. 
- Why is there a difference between the specific contrasts results and the TukeyHSD results?
- Why is it better to specify specific contrasts rather than carry out multiple t-tests?

***
## Multivariate Analyses

[Multivariate analysis](https://en.wikipedia.org/wiki/Multivariate_analysis) is a broad term that includes methods such as **Clustering analysis**, **Principal Component Analysis** (PCA), or **Linear Discriminant Analysis** (LDA).

We will use a data set from my Master Thesis that contains morphological measurements on 10 leaf characters in eight [*Dalbergia*](https://en.wikipedia.org/wiki/Dalbergia) rosewood species from Madagascar. 

Let us read the data and get an overview. Note the use of `row.names` here: it automatically assigns row names of the resulting `data.frame` as the values stored in the column `ID`, and removes `ID` from the set of variables.

In [0]:
d.mis <- read.csv("https://raw.githubusercontent.com/scrameri/Umweltbiologie/master/Multivariate.csv", row.names = "ID")
str(d.mis)

Since the dataset does not have too many variables, we can produce all pairwise plots using `ggparis` from the [*GGally*](https://cran.r-project.org/web/packages/GGally/index.html) package.

In [0]:
gp <- ggpairs(data = d.mis)
print(gp)

You see that some of the variables are highly *correlated* among each other. While this is a potential problem for linear models, this is not a problem for many multivariate techniques.

### Data imputation
Most multivariate techniques cannot handle missing values. Bear this in mind when designing experiments and measuring data, and make sure to collect data that is *as complete as possible*.

One way to get around the missingness problem is to do a **Complete Case Analysis** by removing any individual with missing data. This may lead to loss of many individuals (most of the data) if missingness is distributed randomly accross individuals.

In the case of large datasets such as genetic datasets, one often has little data for some variables (genetic loci) for technical reasons. These loci might however be highly correlated to other loci with less missingness, and can thus be entirely removed from analysis without much information loss. 

However, before you remove any variable with one or more missing values, it makes sense to remove individuals that have few overall data, because that might 'save' many variables from being excluded at the expense of just a few excluded individuals.


Let us check how much missingness our dataset has, using a *self-written* **function** that computes the proportion of missing data.

Think of `x` as a vector of numbers, e.g. one column or one row in the dataset. The missingness is a function of `x`, and computed as the `sum` of `NA` values in `x` divided by the number of observations in `x`. Such a function is only slightly more complex than standard functions such as `mean`, but it could be an arbitrarily complex function with many intermediate steps and variables.

In [0]:
# define the function that computes the percentage of missing values in a numeric vector
missingness <- function(x) {sum(is.na(x))/length(x)}

#### the apply function family
Let us now apply this function to all columns, and all rows, to identify how much missingness these have. For such tasks, we use the `apply` familiy of functions. Have a look at the manuals for `apply`, `sapply`, `lapply` and `tapply`, these can be quite useful and save quite a lot of coding.

The function `apply` takes a `data.frame` `X` as input, a `MARGIN` specification, and *applies* the function specified by the `FUN` argument to *each* element of the data subset defined by `MARGIN`. The output of `apply` can be a single value, a named vector or a matrix, depending on the `MARGIN` and `FUN` employed.

`MARGIN` can be set as
- `1` for "apply function over each row"
- `2` for "apply function over each column"
- `c(1,2)` for "apply function over rows and columns"


By the way: any further arguments that are taken by `FUN` can be passed as additional arguments, as indicated by the `...` in the `apply` function definition. If you wanted to apply the `mean` function to get column means, you could set `FUN = mean, na.rm = TRUE` to override the default behaviour that returns `NA` if there are missing values.

In [0]:
print(apply(X = d.mis[,-c(1:2)], MARGIN = 2, FUN = mean))

In [0]:
print(apply(X = d.mis[,-c(1:2)], MARGIN = 2, FUN = mean, na.rm = TRUE))

You can now combine `apply` with the self-written function `missingness` as follows (once over individuals, once over variables):

In [0]:
mis.rows <- apply(X = d.mis[,-c(1:2)], MARGIN = 1, FUN = missingness) # MARGIN = 1 computes over *rows*
mis.cols <- apply(X = d.mis[,-c(1:2)], MARGIN = 2, FUN = missingness) # MARGIN = 2 computes over *columns*

# this sorts the individual and variable missingness and shows the 10 instances with highest missingness
cat("\nmissingness per individual\n")
print(head(sort(mis.rows, decreasing = TRUE), n = 10))

cat("\nmissingness per variable\n")
print(head(sort(mis.cols, decreasing = TRUE), n = 10))

We see that one variable has nearly 10% missing data, while two individuals have 20% missingness. Excluding one out of 10 variables would throw away much more information than excluding two out of 126 individuals that likely contribute to the observed missingness in `leaflet.lower.hairy`. Before removing the two individuals, check what you are losing in terms of observations per `Species`.

![Question](https://img.icons8.com/flat_round/64/000000/question-mark.png)
- To which `Species` do the two individuals with high missingness belong?
- Can we remove these individuals without losing data for a given `Species`? If yes, create a data.frame where these individuals are excluded.

In [0]:
# write your code here

If the data table still contains missing values after removing individuals or/and variables with little data, these can be replaced by some unbiased values via **data imputation**. There are many sophisticated methods to do this. One simple method is to impute **variable means**.

BEWARE: always bear in mind how much missingness your data has. Everitt & Hothorn (2011) write: 
>"Imputing an observed variable mean for a variable with missing values preserves the observed sample means but distorts the covariance matrix [...], biasing estimated variances and covariances towards zero. [...] one should always bear in mind that the imputed values are not real measurements. We do not get something for nothing! And if there is a substantial proportion of individuals with large amounts of missing data, one should clearly question whether any form of statistical analysis is worth the bother."


![Question](https://img.icons8.com/flat_round/64/000000/question-mark.png)
- Can you write a function that replaces all `NA` values in a vector by the `mean` of that vector? Hint: use the `return` function to return the imputed vector at the end of your function.
- Can you apply that function to all variables with missing data, and assign the output to a `data.frame` named `d.imp`? Hint: apply returns a `matrix` rather than a `data.frame`.

In [0]:
# Write your function and application here

# Test your function on this test vector (yourfunction(tes))
test <- c(1.1,3.2,6.4,NA,3.6)
print(mean(test, na.rm = TRUE))

# Apply your function to all numeric columns of d.mis, and assign the output to d.imp

***
### Data Aggregation
It may be of interest to compute a **summary statistic** such as the `median` of *all* numeric variables *per species*. This would allow for a quick comparison between species.

Of course, one could do subsets of the dataset for each species to calculate the summary statistic, or apply the `tapply` function for each variable, but that takes lines of code for each species or variable, respectively. There is an even more powerful function to do such calculations in *one* line of code. Meet the `aggregate` function: it takes a `data.frame` of numeric variables as input, and applies a specified function `FUN` to *subsets* of each variable. The subsets are defined by the `by` argument, and can include one or more factor variables, which *have* to be specified as a *list*.


In [0]:
aggregate(x = d.mis[,-c(1,2)], by = list(d.mis$Species), FUN = median, na.rm = TRUE)

# It even works if we want to summarize (aggregate) the data by two factors'
aggregate(x = d.mis[,-c(1,2)], by = list(d.mis$Species, d.mis$Region), FUN = median, na.rm = TRUE)

***
### Plot multivariate data
There are a number of special plots designed to visualize large multivariate datasets. One of them is the **Co-Plot**. A Co-Plot displays scatterplots of two variables, but separately for different states of up to two other continuous or discrete variables.

The `coplot` function takes the variables to be displayed as a formula:
- `coplot(y ~ x | separated_by_x + separated_by_y, data = data)`


In [0]:
# color vector for 8 species
col <- c("#A6CEE3","#72B29C","#4F9F3B","#E93E3F","#FDAC4F","#D1AAB7","#A99099","#B15928")

# Co-Plot
coplot(leaflet.shape1 ~ leaflet.length | Species + Region, data = d.mis, 
       col = col[d.mis$Species], pch = 19)  # note the use of col
legend("topright",levels(d.mis$Species),pch = 19, col = col, 
       xpd = TRUE, bty = "n", cex = 0.7)

Now we already see that Species1, 2, 3 and 5 are similar with respect to leaflet.length and leaflet.shape1, or that Species 6 has by far the longest leaflets. But what about overall similarity based on all the data (could be thousands of variables, think e.g. of next-generation sequencing data)? This is a **dimensionality problem**, which is where **Principal Components Analysis** (PCA) can help.

PCA can only handle **numeric** data (continuous or discrete). Let us therefore subset the numeric variables from the imputed dataset.

In [0]:
numericvars <- names(d.mis)[-c(1:2)]
factorvars <- names(d.mis)[1:2]

# define impute function
impute <- function(x) {x[is.na(x)] <- mean(x, na.rm = TRUE) ; return(x)}

# apply impute function and assign data.frame with factor variables
d.imp <- data.frame(d.mis[,factorvars], apply(d.mis[,numericvars], 2, impute))

# subset data.frame to include numeric variables only
d.pca <- d.imp[,numericvars]

### Principal Component Analysis (PCA)

Before PCA is carried out, the dataset is sometimes transformed. Two transformations are important here:
- centering `x <- x-mean(x)`
- scaling `x <- x/sd(x)`

In PCA, all variables are usually **centered** around zero, by subtracting their mean. The question is whether one should **scale** the variables to unit variance, that is, divide them by their standard deviation. 

Rules: 
- if you deal with variables that measure different aspects on *different scales* such as size and shape, these variables will have very *different variance*. If you want to normalize for the different scale, it is very important to scale the variables to unit variance.
- remove any variable with *zero* variance, otherwise prcomp will throw an error.

In [0]:
print(sort(apply(X = d.pca, MARGIN = 2, FUN = var), decreasing = TRUE))


Every variable has variation, but `leaflet.area` has 10^10 times the variance of `leaflet.shape2`. That ist due to the scale of measurement, not due to the amount of information that these variables contain. We will therefore center and scale the dataset for PCA as follows:![alt text](https://)

In [0]:
# Principal Component Analysis (PCA)
pca <- prcomp(x = d.pca, center = TRUE, scale. = TRUE) # don't forget the "." after scale!
# prcomp(x = scale(d.pca, center = TRUE, scale = TRUE), center = FALSE, scale. = FALSE)' # equivalent

# Structure of objects of class 'pca'
str(pca)

In `pca` objects, there are three important results to look at:
- `$x` gives the **PCA scores**, the coordinates of each individual in PCA space
- `$sdev` contains information on the squared variance (**standard deviation**) of each principal component (PC)
- `$rotation` contains **variable loadings**, the information on how much each original variable contributs to each PC

#### The PCA scores
Let us take a look at the PCA scores. Note why it was worth to set the `ID` variable as rownames.

In [0]:
head(pca$x, n = 3)

If you have more individuals than variables, the PCA scores table has dimension [number of individuals, number of variables]. If you had less individuals than variables, the dimensions would be [number of individuals, number of individuals].
  
In total, the information contained in `pca$x` is equivalent to the information in the input data. The difference between the PCA scores and the original data is that the first few PCs contain most of the variation (i.e., information), whereas the informative variation may be scattered throughout all variables in the original data. This **reduction of dimensionality** is one important goal of PCA.

In [0]:
dim(pca$x)

#### The variance explained
In PCA, we want to know the *fraction of variance* contained in (explained by) each principal component. Remember from your statistics course that variance is squared standard deviation. The fraction is simply computed as the component variance divided by the total variance.

In [0]:
# Explained variance
# print(pca$sdev^2) # equivalent to apply(X = pca$x, MARGIN = 2, FUN = var)

# Fraction of explained variance
print(var <- pca$sdev^2 / sum(pca$sdev^2)) # magical formula to get PC variances

# Assign names to the variance vector
names(var) <- paste0("PC", 1:length(var)) # makes a named vector

# Cumulative fractions of variance should add up to 1
print(cumsum(var))

Explained variance is often presented as a **Screeplot**, which is just a barplot of component variances:


In [0]:
# This plot helps to see which PCs contain a lot of information, i.e. are biologically meaningful.
barplot(100*var, xlab = "Principal Component", ylab = "Component Variance (%)")

# This plot helps to decide which PCs explain enough variance to satisfy a certain criterion, e.g. 80% of total variance.
barplot(100*cumsum(var), xlab = "Principal Component", ylab = "Cumulative Component Variance (%)")


#### Variable loadings

The variable loadings are the contribution of each original variable to each Principal Component. These are stored in the rotation matrix `$rotation`. I've written a function that vizualizes the relative importance of each variable to each principal component.

In [0]:
loadings <- data.frame(pca$rotation)

loadingplot <- function(x, component = 1, abs = TRUE) {
  # take absolute values if indicated
  if (abs) {x <- abs(x)}

  # plot
  c <- x[,component]

  ggplot(x, aes(x = seq_along(c), xend = seq_along(c),
                y = 0, yend = c)) +
    geom_segment() +
    geom_text(aes(x = seq_along(c), y = c, 
                  label = rownames(x)), size = 3) +
    scale_x_continuous(breaks = seq_along(c), 
                     labels = seq_along(c)) +
    labs(x = "PC Component", 
         y = paste(ifelse(abs, "Absolute ", ""), "PC Loading (Coefficient)")) +
    ggtitle(paste("Component", component)) +
    theme_bw()
}

loadingplot(loadings, 1, abs = TRUE)
loadingplot(loadings, 2, abs = TRUE)
loadingplot(loadings, 3, abs = TRUE)

![Question](https://img.icons8.com/flat_round/64/000000/question-mark.png)
- What do the first three components 'measure'?

#### The relationship between variable loadings and variable scores
You can think of the PC variable loadings as the **coefficients** that fit the input data to a PC score. In other words: 
> the PC axes are the *linear combination* of the *data vectors* (rows or individuals) with the loadings as coefficients.

The larger the absolute coefficient, the larger the contribution of that variable to the principal component. If a large coefficient is positive, individuals with high values in that variable will have a positive PC axis score.

In [0]:
# define the input to PCA (the scaled and centered variables)
pca.input <- as.matrix(scale(d.pca, center = TRUE, scale = TRUE))

# from the rotation matrix, calculate the coordinate of SH031 on PC1
cat("input data\n")
print(pca.input["SH156",])
cat("\nPC 1 loadings\n")
print(pca$rotation[,1])

# calculate by hand
cat("\nby hand:\n")
print(1.606827*0.3319507 + 2.988947*0.3742269 + 3.849139*0.3881035 + 4.332554*0.3659823 + -1.118697*-0.2518057 + -1.703997*-0.3317078 + -1.526139*-0.3272454 + -2.532783*-0.3065936 + 1.356265*0.1160678 + -2.155491*-0.2778463)

# calculate one PC score by scalar multiplication
cat("\nby scalar multiplication:\n")
print(pca.input["SH156",] %*% pca$rotation[,1])

# calculate all PC scores by matrix multiplication
cat("\nidentity test:\n")
scores <- pca.input %*% pca$rotation
print(all.equal(scores, pca$x)) # if TRUE, you did it!

#### Mathematics of PC loadings
**Component variances** in a PCA on some data are the same as the **eigenvalues** of the **covariance matrix** of the data.

If PCA is carried out on the scaled data, the covariance matrix is transformed to the **correlation matrix** before PCA.

In [0]:
# eigenvalues
cat("Standard deviations of principal components\n")
print(pca$sdev^2)

cat("Eigenvalues of the scaled covariance matrix\n")
print(eigen(cov(scale(d.pca)))$value)

cat("Eigenvalues of the correlation matrix\n")
print(eigen(cor(d.pca))$value)

**PCA loadings** (rotation matrix) on a PCA of some data are the same as the **eigenvectors** of the covariance matrix of the data (sign can be opposite).

In [0]:
# eigenvectors
cat("Principal component loadings\n")
print(head(pca$rotation, n = 3))

cat("\nEigenvectors of the scaled covariance matrix\n")
print(head(eigen(cov(scale(d.pca)))$vectors, n = 3))

cat("\nEigenvectors of the correlation matrix\n")
print(head(eigen(cor(d.pca))$vectors, n = 3))

#### Visualizing PCA results
Finally, we want to see the famous scatterplot of the first PCA components, nicely colored by some covariates. The `factoextra` package offers handy functions to plot both the PCA loadings and the PCA scores.

In [0]:
## PCA variable loadings (here displayed for PC 1 and 2)
fviz_pca_var(pca, col.var = "contrib", axes = c(1,2),
             gradient.cols = c("gray25", "gray45", "red"), repel = TRUE)

## PCA scores (here for PC 1 and 2, with 90% confidence ellipses around Species)
fviz_pca_ind(pca, axes = c(1, 2),
             habillage = d.imp$Species, # habillage is the factor used for colors
             palette = col,
             addEllipses = TRUE, ellipse.level = 0.9, # sets size of ellipses
             ellipse.alpha = 0.5,  # sets ellipse transparency 
             label = "ind", labelsize = 1, # use label = "none" to omit labels
             repel = TRUE) +
  coord_fixed() # sets the same scale for both components

![Question](https://img.icons8.com/flat_round/64/000000/question-mark.png)
By looking at these visualizations only:
- Which variables are highly correlated? Verify using `cor` or `cor.test`.
- Which `Species` has the longest leaves and largest leaflets?
- Which `Species` have hairy leaves and leaflets?

![Question](https://img.icons8.com/flat_round/64/000000/question-mark.png)
Carry out a nested PC analysis within the morphologically similar species 1, 3, 4 and 5, and dispaly the results.
- Can these species be differentiated based on these leaf characters? If so, by which variables?

In [0]:
# subset
d.pca2 <- d.pca[d.imp$Species %in% c("Species1","Species2","Species3","Species5"),]
d.imp2 <- d.imp[d.imp$Species %in% c("Species1","Species2","Species3","Species5"),]

# PCA
pca2 <- prcomp(d.pca2, center = TRUE, scale. = TRUE)

# PCA screeplot
var2 <- pca2$sdev^2/sum(pca2$sdev^2) 
barplot(100*var2, xlab = "Principal Component", ylab = "Component Variance (%)")

# PCA loadings
fviz_pca_var(pca2, col.var = "contrib", axes = c(1,2),
             gradient.cols = c("gray25", "gray45", "red"), repel = TRUE)

# PCA scores of PC1 and PC2
fviz_pca_ind(pca2, axes = c(1, 2),
             habillage = d.imp2$Species, # habillage is the factor used for colors
             palette = col,
             addEllipses = TRUE, ellipse.level = 0.9, # sets size of ellipses
             ellipse.alpha = 0.5,  # sets ellipse transparency 
             label = "ind", labelsize = 1, # use label = "none" to omit labels
             repel = TRUE) +
  coord_fixed() # sets the same scale for both components

***
## References
- G. W. Oehlert (2000) A first course in design and analysis of experiments. Freeman & Co.
- B. Everitt & T. Hothorn (2011) An Introduction to Applied Multivariate Analysis with R. Springer.

### Links
ANOVA Teaching Notes by Dr. Lukas Meier, Senior Scientist at the Seminar for Statistics
- https://stat.ethz.ch/~meier/teaching/anova/